In [13]:
import gym
from gym import spaces
import numpy as np

class TrafficGenerator(gym.Env):
    
    #doesnt really workj
    def getObs(self):
        # Loop over the packet types
        for i in range(len(self.mean_delay)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
            
           
            observation.append([current_length, current_waiting_time])

       
        observation = np.array(observation)
        return observation
    
    def __init__(self):
        
        self.action_space = spaces.Discrete(2)
        
        # Define the observation space (number of packets in each queue and their waiting time)
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(3,2), dtype=int)
        
        # pancket info (DataType, arrival_rate, mean_delay )
        #self.packetInfo = [[0, 0.3, 6],[1, 0.25, 4],[2, 0.4, float('inf')]]
        
        self.dataType = [0, 1, 2]
        self.arrival_rate = [0.3, 0.25, 0.4]
        self.mean_delay_req = [6, 4, float('inf')]
        self.packetInfo = [[elem1, elem2, elem3] for elem1, elem2, elem3 in zip(self.dataType, self.arrival_rate, self.mean_delay_req)]
        self.curr_mean_delay_best_effort = 0
        self.packet = 1
        self.timeslot = 1
        self.totaltime = self.timeslot
        
        # Initialize the queues
        self.queues = [[], [], []]
        
        self.current_queue = 0
      
    
    def step(self, action):  
        self.totaltime += self.timeslot
        print(self.totaltime)
        #packet generator
        for i in range(len(self.packetInfo)):
            for sublist in self.packetInfo:
                if sublist[0] == i and np.random.uniform() < sublist[1]:
                    self.queues[i].append(self.packet)
                    #print("Appending to queue", i)
        
        for i in range(len(self.queues)):
            for j in range(len(self.queues[i])):
                self.queues[i][j] -= self.timeslot
                
        
        # services the queues
        #for i in range(len(self.queues)):
            #print("queue ", i, ": has a length of", len(self.queues[i]))
        #print("ACTION queue ", action, ": has a length of", len(self.queues[action]))
        if action == 0 and len(self.queues[self.current_queue]) > 0:
            self.queues[self.current_queue].pop(0)
            for i in range(len(self.queues[self.current_queue])):
                self.queues[self.current_queue][i] -= 1
        elif action == 1:
            if self.current_queue == 0:
                self.current_queue = 2
            else:
                self.current_queue -= 1
        elif action == 2:
            if self.current_queue == 2:
                self.current_queue = 0
            else:
                self.current_queue += 1
        
        #TODO rewardoptions?
        reward = 1
        
        observation = []
        for i in range(len(self.mean_delay_req)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
    
            observation.append([current_length, current_waiting_time])
            if i == 2:
                #reward option
                if self.curr_mean_delay_best_effort == 0:
                    curr_mean_delay_best_effort = current_waiting_time
                if self.curr_mean_delay_best_effort < current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = 1
                elif self.curr_mean_delay_best_effort > current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = -1
                else:
                    reward = 1
        
                #print(self.curr_mean_delay_best_effort)
                #print(current_waiting_time)
        print(f"observation{observation}")
        if observation[0][1] < -4:
            reward = -10
        elif observation[1][1] < -6:
            reward = -10
        observation = np.array(observation)

         # Loop over the packet types
        """
        observation = []
        for i in range(len(self.mean_delay)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
            
    
            observation.append([current_length, current_waiting_time])

        observation = np.array(observation)
        """
        info = {"mean_delay_0": self.mean_delay_req[0],
        "mean_delay_1": self.mean_delay_req[1],
        "mean_delay_2": self.mean_delay_req[2],
        "arrival_rate_0": self.arrival_rate[0],
        "arrival_rate_1": self.arrival_rate[1],
        "arrival_rate_2": self.arrival_rate[2]}
        done = len(self.queues[0]) + len(self.queues[1]) + len(self.queues[2]) == 0 or (self.totaltime >= 100)
        return observation, reward, done, info
        
    def reset(self):
        # Reset the queues
        self.queues = [[], [], []]
        return np.array([[len(self.queues[i]), 0.0] for i in range(len(self.mean_delay_req))])
        
    def render(self):
        pass

In [15]:
env = TrafficGenerator()
obs = env.reset()
done = False
x = 0
while x < 100:
    print("~~~~~~~~~")
    print(obs[:, 0])
    
    if obs[:, 0][0] > 0 or obs[:, 0][1] > 0:
        action = np.argmax(obs[:2, 0])
    else:
        action = 2 
    
    print("chosen", action)
    obs,reward, done, info = env.step(action)
    print("reward", reward)
    
    x = x +1

~~~~~~~~~
[0. 0. 0.]
chosen 2
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
reward 1
~~~~~~~~~
[0. 0. 0.]
chosen 2
3
observation[[0, 0.0], [0, 0.0], [1, 0.0]]
reward 1
~~~~~~~~~
[0. 0. 1.]
chosen 2
4
observation[[0, 0.0], [0, 0.0], [2, -0.5]]
reward -1
~~~~~~~~~
[0. 0. 2.]
chosen 2
5
observation[[0, 0.0], [1, 0.0], [2, -1.5]]
reward -1
~~~~~~~~~
[0. 1. 2.]
chosen 1
6
observation[[0, 0.0], [1, -1.0], [2, -2.5]]
reward -1
~~~~~~~~~
[0. 1. 2.]
chosen 1
7
observation[[0, 0.0], [1, -2.0], [3, -2.3333333333333335]]
reward 1
~~~~~~~~~
[0. 1. 3.]
chosen 1
8
observation[[0, 0.0], [1, -3.0], [4, -2.5]]
reward -1
~~~~~~~~~
[0. 1. 4.]
chosen 1
9
observation[[0, 0.0], [2, -2.0], [4, -3.5]]
reward -1
~~~~~~~~~
[0. 2. 4.]
chosen 1
10
observation[[0, 0.0], [2, -3.0], [5, -3.6]]
reward -1
~~~~~~~~~
[0. 2. 5.]
chosen 1
11
observation[[0, 0.0], [2, -4.0], [5, -4.6]]
reward -1
~~~~~~~~~
[0. 2. 5.]
chosen 1
12
observation[[0, 0.0], [2, -5.0], [5, -5.6]]
reward -1
~~~~~~~~~
[0. 2. 5.]
chosen 1
13
observation[

# Training the DQN agent dude

In [16]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [17]:
#states = env.observation_space.shape
states = (1,3,2)
actions = env.action_space.n

In [18]:
def build_model(states, actions):
    model = Sequential() 
    model.add(Flatten(input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    model.compile(optimizer=Adam(), loss='mse')
    print(states)
    
    return model

In [19]:
model = build_model(states, actions)


(1, 3, 2)


In [20]:
states

(1, 3, 2)

In [21]:
actions

2

In [22]:
def build_agent(model, actions, states):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [23]:
import tensorflow as tf
import keras
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.12.0
Keras version: 2.12.0


In [25]:
optimizer = Adam(learning_rate=1e-3)
Adam._name = "adam"
dqn = build_agent(model, actions, states)
dqn.compile(optimizer, metrics=['mae'])
dqn.fit(env, nb_steps=1000, visualize=False, verbose=1)

Training for 1000 steps ...
Interval 1 (0 steps performed)
113
observation[[1, 0.0], [0, 0.0], [0, 0.0]]
    1/10000 [..............................] - ETA: 30:05 - reward: 1.0000114
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
115
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
116
observation[[1, 0.0], [0, 0.0], [0, 0.0]]
117
observation[[1, 0.0], [0, 0.0], [0, 0.0]]
118
observation[[1, 0.0], [0, 0.0], [0, 0.0]]
119
observation[[1, 0.0], [0, 0.0], [0, 0.0]]
120
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
121
observation[[1, 0.0], [0, 0.0], [1, 0.0]]
122
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
123
observation[[0, 0.0], [0, 0.0], [0, 0.0]]


AttributeError: 'Adam' object has no attribute 'get_updates'

In [12]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
1102
1
Episode 1: reward: 1.000, steps: 1
1103
1
Episode 2: reward: 1.000, steps: 1
1104
1
Episode 3: reward: 1.000, steps: 1
1105
1
Episode 4: reward: 1.000, steps: 1
1106
1
Episode 5: reward: 1.000, steps: 1
1107
1
Episode 6: reward: 1.000, steps: 1
1108
1
Episode 7: reward: 1.000, steps: 1
1109
1
Episode 8: reward: 1.000, steps: 1
1110
1
Episode 9: reward: 1.000, steps: 1
1111
1
Episode 10: reward: 1.000, steps: 1
1112
1
Episode 11: reward: 1.000, steps: 1
1113
1
Episode 12: reward: 1.000, steps: 1
1114
1
Episode 13: reward: 1.000, steps: 1
1115
1
Episode 14: reward: 1.000, steps: 1
1116
1
Episode 15: reward: 1.000, steps: 1
1117
1
Episode 16: reward: 1.000, steps: 1
1118
1
Episode 17: reward: 1.000, steps: 1
1119
1
Episode 18: reward: 1.000, steps: 1
1120
1
Episode 19: reward: 1.000, steps: 1
1121
1
Episode 20: reward: 1.000, steps: 1
1122
1
Episode 21: reward: 1.000, steps: 1
1123
1
Episode 22: reward: 1.000, steps: 1
1124
1
Episode 23: reward: 1.000, 

In [13]:
print(states)


(1, 3, 2)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 6)                 0         
                                                                 
 dense (Dense)               (None, 24)                168       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 843
Trainable params: 843
Non-trainable params: 0
_________________________________________________________________


In [1]:
model.shape

NameError: name 'model' is not defined